1.Write a Bash script thatg
- Creates a directory named "backup" in the user's home directory^
- Copies all .txt files from the current directory into the "backup" directory^
- Appends the current date and time to the filenames of the copied files.

In [ ]:
#!/bin/bash

BACKUP_DIR="$HOME/backup"

mkdir -p "$BACKUP_DIR"

TIMESTAMP=$(date +"%Y%m%d_%H%M%S")

for file in *.txt; do
    if [ -e "$file" ]; then
        BASENAME=$(basename "$file" .txt)
        # Construct the new filename with timestamp
        NEW_FILENAME="${BASENAME}_${TIMESTAMP}.txt"
        # Copy the file to the backup directory with the new name
        cp "$file" "$BACKUP_DIR/$NEW_FILENAME"
    fi
done

echo "Backup complete. Files copied to $BACKUP_DIR with timestamp."


2.Create a script thatg
- Checks the system’s CPU and memory usage^
- Reports if the CPU usage is above 80% or if the available memory is below 20%^
- Logs the results to a file named system_health.log.

In [ ]:
#!/bin/bash

LOG_FILE="system_health.log"
TIMESTAMP=$(date '+%Y-%m-%d %H:%M:%S')

# Get CPU usage (average over 1 second)
CPU_USAGE=$(top -bn1 | grep "Cpu(s)" | awk '{print 100 - $8}')
CPU_USAGE_INT=${CPU_USAGE%.*}

# Get memory usage
MEMORY=$(free | awk '/Mem:/ {printf("%.0f", $7/$2 * 100)}')  # Available memory %

# Prepare status
STATUS="OK"
if [ "$CPU_USAGE_INT" -gt 80 ] || [ "$MEMORY" -lt 20 ]; then
    STATUS="WARNING"
fi

# Log the result
echo "$TIMESTAMP - CPU: ${CPU_USAGE_INT}% | Available Mem: ${MEMORY}% - $STATUS" >> "$LOG_FILE"

# Optional: print to terminal
echo "$TIMESTAMP - CPU: ${CPU_USAGE_INT}% | Available Mem: ${MEMORY}% - $STATUS"


3.- Reads a list of usernames from a file (e.g., user_list.txt)^
- Creates a new user for each username^
- Generates a random password for each user and saves the username and password to a file named
credentials.txt.

In [ ]:
#!/bin/bash

USER_FILE="user_list.txt"
OUTPUT_FILE="credentials.txt"

# Clear credentials file before starting
> "$OUTPUT_FILE"

# Check if user list file exists
if [ ! -f "$USER_FILE" ]; then
    echo "User list file '$USER_FILE' not found!"
    exit 1
fi

# Loop through each line in the file
while IFS= read -r username || [[ -n "$username" ]]; do
    # Skip empty lines
    [ -z "$username" ] && continue

    # Generate a random 12-character password
    password=$(openssl rand -base64 12)

    # Create user with no home directory and no prompt if already exists
    if id "$username" &>/dev/null; then
        echo "User $username already exists, skipping."
    else
        useradd "$username"
        echo "$username:$password" | chpasswd
        echo "$username:$password" >> "$OUTPUT_FILE"
        echo "User $username created."
    fi
done < "$USER_FILE"

echo "All users processed. Credentials saved in $OUTPUT_FILE."


4.Create a script thatg
- Takes a directory path as input from the user^
- Compresses the directory into a .tar.gz file^
- Saves the compressed file with a name that includes the current date (e.g., backup_2023-08-20.tar.gz).

In [ ]:
#!/bin/bash

# Prompt the user for the directory path
read -p "Enter the path of the directory to compress: " dir_path

# Check if the directory exists
if [ ! -d "$dir_path" ]; then
    echo "Error: Directory does not exist!"
    exit 1
fi

# Get current date
DATE=$(date +%F)  # Format: YYYY-MM-DD

# Get base name of directory (without full path)
DIR_NAME=$(basename "$dir_path")

# Output filename
ARCHIVE_NAME="backup_${DIR_NAME}_${DATE}.tar.gz"

# Create compressed archive
tar -czf "$ARCHIVE_NAME" -C "$(dirname "$dir_path")" "$DIR_NAME"

echo "Directory compressed to: $ARCHIVE_NAME"


5.- Implements a simple command-line to-do list^
- Allows the user to add tasks, view tasks, and remove tasks^
- Saves the tasks to a file (e.g., todo.txt).

In [ ]:
#!/bin/bash

TODO_FILE="todo.txt"

# Ensure the file exists
touch "$TODO_FILE"

# Function to display tasks
view_tasks() {
    if [ ! -s "$TODO_FILE" ]; then
        echo "No tasks found."
    else
        echo "Your To-Do List:"
        nl -w2 -s'. ' "$TODO_FILE"
    fi
}

# Function to add a task
add_task() {
    read -p "Enter your task: " task
    echo "$task" >> "$TODO_FILE"
    echo "Task added."
}

# Function to remove a task by number
remove_task() {
    view_tasks
    read -p "Enter the task number to remove: " num
    if sed -i "${num}d" "$TODO_FILE"; then
        echo "Task removed."
    else
        echo "Invalid task number."
    fi
}

# Menu loop
while true; do
    echo -e "\nTo-Do List Menu"
    echo "1. View tasks"
    echo "2. Add task"
    echo "3. Remove task"
    echo "4. Exit"
    read -p "Choose an option [1-4]: " choice

    case $choice in
        1) view_tasks ;;
        2) add_task ;;
        3) remove_task ;;
        4) echo "Goodbye!"; exit ;;
        *) echo "Invalid option, please try again." ;;
    esac
done


6.Write a script thatg
- Reads a list of software package names from a file (e.g., packages.txt)^
- Installs each package using the appropriate package manager (apt, yum, etc.)^
- Logs the installation status of each package.

In [ ]:
#!/bin/bash

PACKAGE_FILE="packages.txt"
LOG_FILE="install_log.txt"

# Detect package manager
if command -v apt &> /dev/null; then
    PKG_MANAGER="apt"
    INSTALL_CMD="sudo apt-get install -y"
elif command -v yum &> /dev/null; then
    PKG_MANAGER="yum"
    INSTALL_CMD="sudo yum install -y"
elif command -v dnf &> /dev/null; then
    PKG_MANAGER="dnf"
    INSTALL_CMD="sudo dnf install -y"
else
    echo "No supported package manager found (apt, yum, dnf)."
    exit 1
fi

echo "Using package manager: $PKG_MANAGER"
echo "Installation log - $(date)" > "$LOG_FILE"

# Read and install packages
while read -r package; do
    [ -z "$package" ] && continue  # Skip empty lines
    echo "Installing: $package"
    if $INSTALL_CMD "$package" &>> "$LOG_FILE"; then
        echo "$package: SUCCESS" >> "$LOG_FILE"
    else
        echo "$package: FAILED" >> "$LOG_FILE"
    fi
done < "$PACKAGE_FILE"

echo "Installation complete. See $LOG_FILE for details."


7.- Takes a text file as input^
- Counts and displays the number of lines, words, and characters in the file^
- Finds and displays the longest word in the file.

In [ ]:
#!/bin/bash

# Ask for the filename
read -p "Enter the name of the text file: " FILE

# Check if file exists
if [ ! -f "$FILE" ]; then
    echo "File not found!"
    exit 1
fi

# Count lines, words, and characters
LINES=$(wc -l < "$FILE")
WORDS=$(wc -w < "$FILE")
CHARS=$(wc -m < "$FILE")

# Find the longest word
LONGEST_WORD=$(tr -cs '[:alnum:]' '[\n*]' < "$FILE" | awk '{ if (length > max) { max = length; word = $0 } } END { print word }')

# Display results
echo "File: $FILE"
echo "Lines: $LINES"
echo "Words: $WORDS"
echo "Characters: $CHARS"
echo "Longest word: $LONGEST_WORD"
